### Class and Static Methods

Asd we saw, when we define a function inside a class, how it behaves (as a function or a method) depends on how the function is accessed: from the class, or from the instance. (We'll cover how that works when we look at descriptors later in this course).

In [1]:
class Person:
    def hello(arg='default'):
        print(f'Hello, with arg={arg}')

If we call `hello` from the class:

In [2]:
Person.hello()

Hello, with arg=default


You'll notice that `hello` was called without any arguments, in fact, `hello` is a regular function:

In [3]:
Person.hello

<function __main__.Person.hello(arg='default')>

But if we call `hello` from an instance, things are different:

In [4]:
p = Person()

In [5]:
p.hello

<bound method Person.hello of <__main__.Person object at 0x7f8f287fb860>>

In [6]:
p.hello()

Hello, with arg=<__main__.Person object at 0x7f8f287fb860>


In [7]:
hex(id(p))

'0x7f8f287fb860'

And as you can see the instance `p` was passed as an argument to `hello`. 

Sometimes however, we define functions in a class that do not interact with the instance itself, but may need something from the class. In those cases, we want the class to be passed to the function as an argument, whether it is called from the class or from an instance of the class.

These are called **class methods**. You'll note that the behavior needs to be different - we don't want the instance to be passed to the function when called from an instance, we want the **class** to be passed to it. In addition, when called from the class, we **also** want the class to be passed to it (this is similar to `static` methods in Java, not to be confused with, as we'll see in a bit, static methods in Python).

We use the `@classmethod` decorator to define class methods, and the first argument of these methods will always be the class where the method is defined.

Let's see a simple example first:

In [8]:
class MyClass:
    def hello():
        # this IS an instance method, we just forgot to add a parameter to capture the instance
        # when this is called from an instance - so this will fail
        print('hello...')
        
    def instance_hello(arg):
        print(f'hello from {arg}')
        
    @classmethod
    def class_hello(arg):
        print(f'hello from {arg}')
        

In [9]:
m = MyClass()

In [10]:
MyClass.hello()

hello...


But, as expected, this won't work:

In [11]:
try:
    m.hello()
except TypeError as ex:
    print(ex)

hello() takes 0 positional arguments but 1 was given


On the other hand, notice now the instance method when called from the instance and the class:

In [12]:
m.instance_hello()

hello from <__main__.MyClass object at 0x7f8ed87fff60>


In [13]:
try:
    MyClass.instance_hello()
except TypeError as ex:
    print(ex)

instance_hello() missing 1 required positional argument: 'arg'


As you can see, the instance method needs to be called from the instance. If we call it from the class, no argument is passed to the function, so we end up with an exception.

This is not the case with class methods - whether we call the method from the class, or the instance, that first argument will always be provided by Python, and will be the class object (not the instance).

Notice how the bindings are different:

In [14]:
MyClass.class_hello

<bound method MyClass.class_hello of <class '__main__.MyClass'>>

In [15]:
m.class_hello

<bound method MyClass.class_hello of <class '__main__.MyClass'>>

As you can see in both these cases, `class_hello` is bound to the class.

But with an instance method, the bindings behave differently:

In [16]:
MyClass.instance_hello

<function __main__.MyClass.instance_hello(arg)>

In [17]:
m.instance_hello

<bound method MyClass.instance_hello of <__main__.MyClass object at 0x7f8ed87fff60>>

So, whenever we call `class_hello` the method is bound to the **class**, and the first argument is the class:

In [18]:
MyClass.class_hello()

hello from <class '__main__.MyClass'>


In [19]:
m.class_hello()

hello from <class '__main__.MyClass'>


Although in this example I used `arg` as the parameter name in our methods, the normal **convention** is to use `self` and `cls` - that way everyone knows what we're talking about!

We sometimes also want to define functions in a class and always have them be just that - functions, never bound to either the class or the instance, however we call them. Often we do this because we need to utility function that is specific to our class, and we want to keep our class self-contained, or maybe we're writing a library of functions (though modules and packages may be more appropriate for this).

These are called **static** methods. (So be careful here, Python static methods and Java static methods do not have the same meaning!)

We can define static methods using the `@staticmethod` decorator:

In [20]:
class MyClass:
    def instance_hello(self):
        print(f'Instance method bound to {self}')
        
    @classmethod
    def class_hello(cls):
        print(f'Class method bound to {cls}')
        
    @staticmethod
    def static_hello():
        print('Static method not bound to anything')

In [21]:
m = MyClass()

In [22]:
m.instance_hello()

Instance method bound to <__main__.MyClass object at 0x7f8ed8811a58>


In [23]:
MyClass.class_hello()

Class method bound to <class '__main__.MyClass'>


In [24]:
m.class_hello()

Class method bound to <class '__main__.MyClass'>


And the static method can be called either from the class or the instance, but is never bound:

In [25]:
MyClass.static_hello

<function __main__.MyClass.static_hello()>

In [26]:
m.static_hello

<function __main__.MyClass.static_hello()>

In [27]:
MyClass.static_hello()

Static method not bound to anything


In [28]:
m.static_hello()

Static method not bound to anything


#### Example

Let's see a more concrete example of using these different method types.

We're going to create a `Timer` class that will allow us to get the current time (in both UTC and some timezone), as well as record start/stop times.

We want to have the same timezone for all instances of our `Timer` class with an easy way to change the timezone for all instances when needed.

If you need to work with timezones, I recommend you use the `pyrz` 3rd party library. Here, I'll just use the standard library, which is definitely not as easy to use as `pytz`.

In [29]:
from datetime import datetime, timezone, timedelta

class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)

So `tz` is a class attribute, and we can set it using a class method `set_timezone` - any instances will share the same `tz` value (unless we override it at the instance level)

In [30]:
Timer.set_tz(-7, 'MST')

In [31]:
Timer.tz

datetime.timezone(datetime.timedelta(-1, 61200), 'MST')

In [32]:
t1 = Timer()
t2 = Timer()

In [33]:
t1.tz, t2.tz

(datetime.timezone(datetime.timedelta(-1, 61200), 'MST'),
 datetime.timezone(datetime.timedelta(-1, 61200), 'MST'))

In [34]:
Timer.set_tz(-8, 'PST')

In [35]:
t1.tz, t2.tz

(datetime.timezone(datetime.timedelta(-1, 57600), 'PST'),
 datetime.timezone(datetime.timedelta(-1, 57600), 'PST'))

Next we want a function to return the current UTC time. Obviously this has nothing to do with either the class or the instance, so it is a prime candidate for a static method:

In [36]:
class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    @staticmethod
    def current_dt_utc():
        return datetime.now(timezone.utc)
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)

In [37]:
Timer.current_dt_utc()

datetime.datetime(2019, 6, 2, 23, 25, 59, 714761, tzinfo=datetime.timezone.utc)

In [38]:
t = Timer()

In [39]:
t.current_dt_utc()

datetime.datetime(2019, 6, 2, 23, 25, 59, 723565, tzinfo=datetime.timezone.utc)

Next we want a method that will return the current time based on the set time zone. Obviously the time zone is a class variable, so we'll need to access that, but we don't need any instance data, so this is a prime candidate for a class method:

In [40]:
class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    @staticmethod
    def current_dt_utc():
        return datetime.now(timezone.utc)
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)
        
    @classmethod
    def current_dt(cls):
        return datetime.now(cls.tz)

In [41]:
Timer.current_dt_utc(), Timer.current_dt()

(datetime.datetime(2019, 6, 2, 23, 25, 59, 733420, tzinfo=datetime.timezone.utc),
 datetime.datetime(2019, 6, 2, 23, 25, 59, 733423, tzinfo=datetime.timezone.utc))

In [42]:
t1 = Timer()
t2 = Timer()

In [43]:
t1.current_dt_utc(), t1.current_dt()

(datetime.datetime(2019, 6, 2, 23, 25, 59, 741248, tzinfo=datetime.timezone.utc),
 datetime.datetime(2019, 6, 2, 23, 25, 59, 741251, tzinfo=datetime.timezone.utc))

In [44]:
t2.current_dt()

datetime.datetime(2019, 6, 2, 23, 25, 59, 745699, tzinfo=datetime.timezone.utc)

And if we change the time zone (we can do so either via the class or the instance, no difference, since the `set_tz` method is always bound to the class):

In [45]:
t2.set_tz(-7, 'MST')

In [46]:
Timer.__dict__

mappingproxy({'__module__': '__main__',
              'tz': datetime.timezone(datetime.timedelta(-1, 61200), 'MST'),
              'current_dt_utc': <staticmethod at 0x7f8ed8836d30>,
              'set_tz': <classmethod at 0x7f8ed8836d68>,
              'current_dt': <classmethod at 0x7f8ed8836da0>,
              '__dict__': <attribute '__dict__' of 'Timer' objects>,
              '__weakref__': <attribute '__weakref__' of 'Timer' objects>,
              '__doc__': None})

In [47]:
Timer.current_dt_utc(), Timer.current_dt(), t1.current_dt(), t2.current_dt()

(datetime.datetime(2019, 6, 2, 23, 25, 59, 761523, tzinfo=datetime.timezone.utc),
 datetime.datetime(2019, 6, 2, 16, 25, 59, 761526, tzinfo=datetime.timezone(datetime.timedelta(-1, 61200), 'MST')),
 datetime.datetime(2019, 6, 2, 16, 25, 59, 761526, tzinfo=datetime.timezone(datetime.timedelta(-1, 61200), 'MST')),
 datetime.datetime(2019, 6, 2, 16, 25, 59, 761527, tzinfo=datetime.timezone(datetime.timedelta(-1, 61200), 'MST')))

So far we have not needed any instances to work with this class!

Now we're going to add functionality to start/stop a timer. Obviously we want this to be instance based, since we want to be able to create multiple timers.

In [48]:
class TimerError(Exception):
    """A custom exception used for Timer class"""
    # (since """...""" is a statement, we don't need to pass)
    
class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    def __init__(self):
        # use these instance variables to keep track of start/end times
        self._time_start = None
        self._time_end = None
        
    @staticmethod
    def current_dt_utc():
        """Returns non-naive current UTC"""
        return datetime.now(timezone.utc)
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)
        
    @classmethod
    def current_dt(cls):
        return datetime.now(cls.tz)
    
    def start(self):
        # internally we always non-naive UTC
        self._time_start = self.current_dt_utc()
        self._time_end = None
        
    def stop(self):
        if self._time_start is None:
            # cannot stop if timer was not started!
            raise TimerError('Timer must be started before it can be stopped.')
        self._time_end = self.current_dt_utc()
        
    @property
    def start_time(self):
        if self._time_start is None:
            raise TimerError('Timer has not been started.')
        # since tz is a class variable, we can just as easily access it from self
        return self._time_start.astimezone(self.tz)  
        
    @property
    def end_time(self):
        if self._time_end is None:
            raise TimerError('Timer has not been stopped.')
        return self._time_end.astimezone(self.tz)
    
    @property
    def elapsed(self):
        if self._time_start is None:
            raise TimerError('Timer must be started before an elapsed time is available')
            
        if self._time_end is None:
            # timer has not ben stopped, calculate elapsed between start and now
            elapsed_time = self.current_dt_utc() - self._time_start
        else:
            # timer has been stopped, calculate elapsed between start and end
            elapsed_time = self._time_end - self._time_start
            
        return elapsed_time.total_seconds()

In [49]:
from time import sleep

t1 = Timer()
t1.start()
sleep(2)
t1.stop()
print(f'Start time: {t1.start_time}')
print(f'End time: {t1.end_time}')
print(f'Elapsed: {t1.elapsed} seconds')

Start time: 2019-06-02 23:25:59.777250+00:00
End time: 2019-06-02 23:26:01.781431+00:00
Elapsed: 2.004181 seconds


In [50]:
t2 = Timer()
t2.start()
sleep(3)
t2.stop()
print(f'Start time: {t2.start_time}')
print(f'End time: {t2.end_time}')
print(f'Elapsed: {t2.elapsed} seconds')

Start time: 2019-06-02 23:26:01.787596+00:00
End time: 2019-06-02 23:26:04.792814+00:00
Elapsed: 3.005218 seconds


So our timer works. Furthermore, we want to use `MST` throughout our application, so we'll set it, and since it's a class level attribute, we only need to change it once:

In [51]:
Timer.set_tz(-7, 'MST')

In [52]:
print(f'Start time: {t1.start_time}')
print(f'End time: {t1.end_time}')
print(f'Elapsed: {t1.elapsed} seconds')

Start time: 2019-06-02 16:25:59.777250-07:00
End time: 2019-06-02 16:26:01.781431-07:00
Elapsed: 2.004181 seconds


In [53]:
print(f'Start time: {t2.start_time}')
print(f'End time: {t2.end_time}')
print(f'Elapsed: {t2.elapsed} seconds')

Start time: 2019-06-02 16:26:01.787596-07:00
End time: 2019-06-02 16:26:04.792814-07:00
Elapsed: 3.005218 seconds
